In [1]:
import os, sys
import numpy as np
sys.path.append('../')

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
from onsstove.onsstove import OnSSTOVE
from onsstove.layer import RasterLayer, VectorLayer

In [4]:
nepal = OnSSTOVE(project_crs=3857, cell_size=(1000, 1000))
output_directory = 'results'
nepal.output_directory = output_directory

In [5]:
path = r"C:\Users\camilorg\Box Sync\Clean cooking Africa paper\05. OnSSTOVE test case\GQ_test_file_social_specs.csv"
nepal.read_scenario_data(path)

In [6]:
path = r"C:\Users\camilorg\Box Sync\Clean cooking Africa paper\05. OnSSTOVE test case\GQ_test_file_tech_specs.csv"
nepal.read_tech_data(path)

In [7]:
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Administrative\npl_admbnda_nd_20201117_shp\npl_admbnda_adm0_nd_20201117.shp"
mask_layer = VectorLayer('vector', 'adm_0', layer_path=path)
os.makedirs(f'{output_directory}vector/adm_0', exist_ok=True)
mask_layer.reproject(3857, f'{output_directory}vector/adm_0')
nepal.mask_layer = mask_layer
# add admin_1 or 2 or 3 layer to gdf here

In [8]:
path = r"data\population_npl_2018-10-01.tif"
os.makedirs(f'{output_directory}/rasters/population', exist_ok=True)
population = RasterLayer('rasters', 'population', layer_path=path, resample='sum')
population.reproject(3857, f'{output_directory}/rasters/population', 1000, 1000)
population.mask(nepal.mask_layer.layer, f'{output_directory}/rasters/population')
nepal.base_layer = population
nepal.population_to_dataframe(population)

In [9]:
np.nansum(population.layer)

28452301.739626173

In [10]:
nepal.calibrate_pop()

In [11]:
nepal.calibrate_urban()

In [12]:
rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc = nepal.base_fuel.relative_risk()

In [13]:
print(rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc)

3.379491698111922 2.6016762384532757 2.9461880136695875 3.807237414338239


In [14]:
paf_0_alri = nepal.base_fuel.paf(rr_0_alri, 1 - nepal.specs['clean_cooking_access'])
paf_0_copd = nepal.base_fuel.paf(rr_0_copd, 1 - nepal.specs['clean_cooking_access'])
paf_0_lc  = nepal.base_fuel.paf(rr_0_ihd, 1 - nepal.specs['clean_cooking_access'])
paf_0_ihd = nepal.base_fuel.paf(rr_0_lc, 1 - nepal.specs['clean_cooking_access'])

In [15]:
nepal.base_fuel.carb(nepal.gdf)

In [16]:
friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_walking_only_friction_surface\2020_walking_only_friction_surface.geotiff"
forest_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\Forest cover\Global Forest Cover Change (GFCC).tif"

nepal.base_fuel.total_time(nepal.specs, friction_path, forest_path, population.path, output_directory)

In [17]:
nepal.raster_to_dataframe(nepal.base_fuel.total_time_yr, name='base_fuel_time', method='read')

In [61]:
nepal.raster_to_dataframe(nepal.base_fuel.travel_time, name='base_fuel_travel_time', method='read')

In [18]:
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\Nepal_DL0\Nepal_DL0.shp"
mv_lines = VectorLayer('vector', 'MV_line', layer_path=path)
os.makedirs(f'{output_directory}/vector/MV_line', exist_ok=True)
mv_lines.reproject(3857, f'{output_directory}/vector/MV_line')

path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\HV-network\HV_lines_georeferenced.shp"
hv_lines = VectorLayer('vector', 'HV_line', layer_path=path)
os.makedirs(f'{output_directory}/vector/HV_line', exist_ok=True)
hv_lines.reproject(3857, f'{output_directory}/vector/HV_line')

In [19]:
nepal.distance_to_electricity(hv_lines=hv_lines, mv_lines=mv_lines)

In [20]:
path = r"data\nighttime_lights.tif"
os.makedirs(f'{output_directory}rasters/Night_lights', exist_ok=True)
ntl = RasterLayer('rasters', 'Night_lights', layer_path=path, resample='average')
ntl.align(population.path, f'{output_directory}/rasters/Night_lights')
ntl.layer[np.isnan(ntl.layer)] = 0 # hey lets make sure that we interpolate NaNs

nepal.raster_to_dataframe(ntl.layer, name='Night_lights', method='read')

In [21]:
nepal.elec_current()

We have identified the existence of transformers or MV lines as input data; therefore we proceed using those for the calibration


In [22]:
# nepal.gdf.crs = 3857
# nepal.gdf.to_file('nepal/results.gpkg', driver='GPKG')

In [23]:
nepal.gdf['Elec_pop_calib'].sum() / nepal.gdf['Calibrated_pop'].sum()

0.5201280449140765

In [24]:
nepal.techs['Electricity'].morbidity(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)

In [25]:
nepal.techs['Electricity'].morbidity

18788130.060000986

In [26]:
nepal.techs['Electricity'].mortality(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)

In [27]:
nepal.techs['Electricity'].mortality

427020517.52045035

In [29]:
nepal.techs['Electricity'].deaths_avoided

13653.562187748523

In [30]:
nepal.techs['Electricity'].carbon_emissions(nepal.specs, nepal.gdf, nepal.base_fuel.carbon)

In [31]:
nepal.techs['Electricity'].decreased_carbon_emissions

0         1.086395
1         1.629592
2         0.192308
3         0.543197
4         0.543197
           ...    
96717     3.416899
96718    18.697766
96719    10.998686
96720    22.405873
96721     4.691313
Name: Households, Length: 96722, dtype: float64

In [32]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Demand\Wealth Index\Wealth index 2011.tif"
wealth_index = RasterLayer('rasters', 'Wealth_index', layer_path=path,  resample='average')
os.makedirs(f'{output_directory}rasters/Wealth_index', exist_ok=True)
wealth_index.align(population.path, f'{output_directory}/rasters/Wealth_index')

In [33]:
nepal.get_value_of_time(wealth_index)

In [34]:
nepal.techs['Electricity'].time_saved(nepal.gdf, nepal.specs)

In [36]:
nepal.techs['Electricity'].time_value.sum()

1043808521.253551

In [41]:
nepal.techs['Electricity'].total_time_saved.sum()

149663438.6670127

In [69]:
nepal.gdf

,geometry,Pop,Calibrated_pop,IsUrban,Households,base_fuel_time,HV_line_dist,MV_line_dist,Night_lights,Elec_pop_calib,Current_elec,Elec_dist,Final_Elec_Code2020,value_of_time,base_fuel_travel_time
0,POINT (9090104.530 3543192.836),31.171455,31.344224,0,5.498987,2993.471554,58.872746,40.521599,0.000000,0.000000,0,40.521599,99,0.058779,3.001292
1,POINT (9090104.530 3536192.836),46.757183,47.016337,0,8.248480,3442.932033,52.392746,33.837849,0.000000,0.000000,0,33.837849,99,0.059024,4.232691
2,POINT (9083104.530 3535192.836),5.517799,5.548382,0,0.973400,2239.061114,54.708317,35.846897,0.000000,0.000000,0,35.846897,99,0.059683,0.934414
3,POINT (9084104.530 3535192.836),15.585728,15.672112,0,2.749493,2280.672201,54.203320,35.355339,0.000000,0.000000,0,35.355339,99,0.059637,1.048417
4,POINT (9064104.530 3534192.836),15.585728,15.672112,0,2.749493,2972.034660,64.536812,47.634022,0.000000,0.000000,0,47.634022,99,0.064742,2.942561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96717,POINT (9724104.530 3044192.836),98.039602,98.582991,0,17.295262,7652.475230,13.038404,1.414214,0.000000,98.582991,1,1.414214,1,NaN,15.765686
96718,POINT (9797104.530 3044192.836),536.486856,539.460361,1,94.642169,3599.900735,17.029387,0.000000,0.000000,539.460361,1,0.000000,1,NaN,4.662742
96719,POINT (9798104.530 3044192.836),315.580504,317.329624,0,55.671864,3660.375916,17.117243,1.000000,0.000000,317.329624,1,1.000000,1,NaN,4.828427
96720,POINT (9722104.530 3043192.836),642.881959,646.445163,1,113.411432,7919.425593,11.000000,0.000000,0.891932,NaN,1,0.000000,1,NaN,16.497056


In [70]:
nepal.techs['Electricity'].discounted_om(nepal.gdf, nepal.specs)

In [71]:
nepal.techs['Electricity'].discounted_om_costs

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
96717    0.0
96718    0.0
96719    0.0
96720    0.0
96721    0.0
Name: Households, Length: 96722, dtype: float64

In [72]:
nepal.techs['Electricity'].discounted_inv(nepal.gdf, nepal.specs)

In [76]:
nepal.techs['Electricity'].discounted_investments

0         302.444271
1         453.666406
2          53.537020
3         151.222135
4         151.222135
            ...     
96717     951.239385
96718    5205.319271
96719    3061.952512
96720    6237.628769
96721    1306.026717
Name: Households, Length: 96722, dtype: float64

In [77]:
nepal.techs['Electricity'].discounted_meals(nepal.gdf, nepal.specs)

In [78]:
nepal.techs['Electricity'].discounted_energy

0         25485.885291
1         38228.827936
2          4511.371144
3         12742.942645
4         12742.942645
             ...      
96717     80157.503996
96718    438633.436558
96719    258019.668564
96720    525622.425941
96721    110054.149870
Name: Households, Length: 96722, dtype: float64

In [108]:
nepal.techs['Electricity'].discount_fuel_cost(nepal.gdf, nepal.specs)

In [110]:
nepal.techs['Electricity'].discounted_fuel_cost

0         417.685342
1         626.528013
2          73.936360
3         208.842671
4         208.842671
            ...     
96717    1313.692427
96718    7188.714655
96719    4228.655679
96720    8614.367536
96721    1803.665234
Length: 96722, dtype: float64

In [111]:
nepal.techs['Electricity'].salvage(nepal.gdf, nepal.specs)

In [112]:
nepal.techs['Electricity'].discounted_salvage_cost

0         272.199844
1         408.299765
2          48.183318
3         136.099922
4         136.099922
            ...     
96717     856.115446
96718    4684.787344
96719    2755.757261
96720    5613.865892
96721    1175.424045
Name: Households, Length: 96722, dtype: float64

In [113]:
nepal.techs['Electricity'].net_benefit(nepal.gdf)

In [117]:
nepal.gdf.loc[nepal.gdf['final_tech']=='electricity']

,geometry,Pop,Calibrated_pop,IsUrban,Households,base_fuel_time,HV_line_dist,MV_line_dist,Night_lights,Elec_pop_calib,Current_elec,Elec_dist,Final_Elec_Code2020,value_of_time,base_fuel_travel_time,net_benefit_electricity,net_benefit_biomass,final_tech,maximum_net_benefit
7,POINT (9083104.530 3534192.836),212.682386,213.861189,0,37.519507,2070.014190,53.851646,34.985710,0.000000,0.000000,0,34.985710,99,0.059703,0.471272,6137.118836,0,electricity,3904.030423
12,POINT (9067104.530 3532192.836),218.200186,219.409571,0,38.492907,2406.853050,62.225395,44.045429,0.000000,0.000000,0,44.045429,99,0.064394,1.394118,7356.717504,0,electricity,5180.099365
14,POINT (9074104.530 3532192.836),249.371641,250.753795,0,43.991894,3364.561626,57.489128,38.897301,0.000000,0.000000,0,38.897301,99,0.061839,4.017977,10834.834979,0,electricity,8930.296156
20,POINT (9122104.530 3527192.836),142.632530,143.423078,0,25.161943,5772.149808,40.261642,28.160255,0.000000,0.000000,0,28.160255,99,0.058356,10.614109,9509.498308,0,electricity,6179.684834
29,POINT (9004104.530 3520192.836),165.469121,166.386242,0,29.190569,2115.388250,34.132095,34.828148,0.000000,0.000000,0,34.828148,99,0.063713,0.595584,5005.285384,0,electricity,2135.025709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96708,POINT (9721104.530 3045192.836),314.029200,315.769722,0,55.398197,7687.900771,10.198039,1.000000,0.149881,315.769722,1,1.000000,1,0.105401,15.862742,45025.737368,0,electricity,43513.340075
96709,POINT (9722104.530 3045192.836),61.131198,61.470021,0,10.784214,7627.425590,11.180340,1.000000,0.374121,61.470021,1,1.000000,1,0.104375,15.697056,8621.003471,0,electricity,851.794415
96710,POINT (9723104.530 3045192.836),869.070444,873.887309,1,153.313563,7566.950409,12.165525,0.000000,0.798194,NaN,1,0.000000,1,0.101040,15.531371,118144.221020,0,electricity,117597.743092
96716,POINT (9723104.530 3044192.836),159.314352,160.197360,0,28.104800,7712.950411,12.041595,1.000000,0.319198,160.197360,1,1.000000,1,0.102936,15.931371,22439.345090,0,electricity,19458.198444


In [89]:
nepal.gdf['net_benefit_biomass'] = 0

In [94]:
nepal.maximum_net_benefit()

In [95]:
nepal.gdf

,geometry,Pop,Calibrated_pop,IsUrban,Households,base_fuel_time,HV_line_dist,MV_line_dist,Night_lights,Elec_pop_calib,Current_elec,Elec_dist,Final_Elec_Code2020,value_of_time,base_fuel_travel_time,net_benefit_electricity,net_benefit_biomass,final_tech,maximum_net_benefit
0,POINT (9090104.530 3543192.836),31.171455,31.344224,0,5.498987,2993.471554,58.872746,40.521599,0.000000,0.000000,0,40.521599,99,0.058779,3.001292,-14044.820276,0,biomass,0.000000
1,POINT (9090104.530 3536192.836),46.757183,47.016337,0,8.248480,3442.932033,52.392746,33.837849,0.000000,0.000000,0,33.837849,99,0.059024,4.232691,-8146.980364,0,biomass,0.000000
2,POINT (9083104.530 3535192.836),5.517799,5.548382,0,0.973400,2239.061114,54.708317,35.846897,0.000000,0.000000,0,35.846897,99,0.059683,0.934414,-85905.526249,0,biomass,0.000000
3,POINT (9084104.530 3535192.836),15.585728,15.672112,0,2.749493,2280.672201,54.203320,35.355339,0.000000,0.000000,0,35.355339,99,0.059637,1.048417,-29988.831976,0,biomass,0.000000
4,POINT (9064104.530 3534192.836),15.585728,15.672112,0,2.749493,2972.034660,64.536812,47.634022,0.000000,0.000000,0,47.634022,99,0.064742,2.942561,-29845.277857,0,biomass,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96717,POINT (9724104.530 3044192.836),98.039602,98.582991,0,17.295262,7652.475230,13.038404,1.414214,0.000000,98.582991,1,1.414214,1,NaN,15.765686,NaN,0,biomass,0.000000
96718,POINT (9797104.530 3044192.836),536.486856,539.460361,1,94.642169,3599.900735,17.029387,0.000000,0.000000,539.460361,1,0.000000,1,NaN,4.662742,NaN,0,biomass,0.000000
96719,POINT (9798104.530 3044192.836),315.580504,317.329624,0,55.671864,3660.375916,17.117243,1.000000,0.000000,317.329624,1,1.000000,1,NaN,4.828427,NaN,0,biomass,0.000000
96720,POINT (9722104.530 3043192.836),642.881959,646.445163,1,113.411432,7919.425593,11.000000,0.000000,0.891932,NaN,1,0.000000,1,NaN,16.497056,NaN,0,biomass,0.000000
